## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-17-02-16-25 +0000,nyt,Hiker Found Dead in Cascade Mountains of Washi...,https://www.nytimes.com/2025/09/16/us/hiker-de...
1,2025-09-17-02-15-13 +0000,nypost,ICE receives more than 150K job applications a...,https://nypost.com/2025/09/16/us-news/ice-rece...
2,2025-09-17-02-12-00 +0000,startribune,Democrat Xp Lee wins special election to succe...,https://www.startribune.com/voters-head-to-the...
3,2025-09-17-02-08-27 +0000,nypost,Sam Ponder’s DMs ‘more disturbing than ever’ a...,https://nypost.com/2025/09/16/sports/sam-ponde...
4,2025-09-17-02-03-46 +0000,bbc,Canadian writer Robert Munsch approved for ass...,https://www.bbc.com/news/articles/c931y04jg1vo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2317/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,52
35,kirk,34
72,gaza,25
34,charlie,24
111,new,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
76,2025-09-16-21-45-00 +0000,wsj,In her first public comments since Trump fired...,https://www.wsj.com/economy/jobs/fired-bls-chi...,127
240,2025-09-16-14-44-00 +0000,wsj,FBI Director Kash Patel is defending his work ...,https://www.wsj.com/politics/policy/fbi-direct...,117
150,2025-09-16-18-31-12 +0000,bbc,Trump says he will sue New York Times for $15bn,https://www.bbc.com/news/articles/cevz419pk8vo...,106
158,2025-09-16-18-03-09 +0000,nypost,Bill Ackman denies Candace Owens’ claim he sta...,https://nypost.com/2025/09/16/media/bill-ackma...,106
334,2025-09-16-08-49-51 +0000,cbc,Trump launches $15B US defamation lawsuit agai...,https://www.cbc.ca/news/world/trump-lawsuit-ne...,105


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
76,127,2025-09-16-21-45-00 +0000,wsj,In her first public comments since Trump fired...,https://www.wsj.com/economy/jobs/fired-bls-chi...
240,109,2025-09-16-14-44-00 +0000,wsj,FBI Director Kash Patel is defending his work ...,https://www.wsj.com/politics/policy/fbi-direct...
170,100,2025-09-16-17-37-00 +0000,wsj,Israel Launches New Ground Offensive Seeking t...,https://www.wsj.com/world/middle-east/israel-l...
111,95,2025-09-16-20-14-00 +0000,wsj,Democrats are introducing a bill in the Senate...,https://www.wsj.com/economy/central-banking/se...
272,52,2025-09-16-13-19-35 +0000,nypost,DA Fani Willis loses bid to appeal Trump’s Geo...,https://nypost.com/2025/09/16/us-news/da-fani-...
244,50,2025-09-16-14-41-00 +0000,cbc,New York judge tosses terrorism charges agains...,https://www.cbc.ca/news/judge-dismisses-terror...
131,46,2025-09-16-19-22-57 +0000,bbc,Eritrean man wins block on removal to France u...,https://www.bbc.com/news/articles/c1dqe2443l1o...
72,40,2025-09-16-21-59-00 +0000,wsj,The House plans to vote on a series of bills t...,https://www.wsj.com/politics/policy/house-repu...
45,38,2025-09-16-23-09-02 +0000,startribune,Charges: Minneapolis man confessed to killing ...,https://www.startribune.com/charges-minneapoli...
260,35,2025-09-16-13-55-32 +0000,nypost,Bondi takes heat from the right after vowing t...,https://nypost.com/2025/09/16/us-news/bondi-ta...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
